### Создание класса



Создайте класс с именем MyKNNReg. Данный класс при инициализации должен принимать один параметр:

k – кол-во ближайших соседей, которые будем рассматривать при подсчете ответа
По-умолчанию: 3
Все переданные (или дефолтные) параметры должны быть сохранены внутри класса.

При обращении к экземпляру класса (или при передачи его в функцию print) необходимо распечатать строку по следующему шаблону:

MyKNNReg class: k=<k>

In [ ]:
class MyKNNReg():

    def __init__(self, k=3):
        self.k = k

    def __str__(self):
        params = (', ').join(f'{key}={value}' for key, value in self.__dict__.items())
        return f'{__class__.__name__} class: {params}'

### Метод fit()

Доработайте класс MyKNNReg следующим образом:

В инициализатор класса добавьте переменную train_size. В ней будет храниться размер обучающей выборки.
Добавьте в класс метод fit. Данный метод должен делать следующее:
На вход принимать две переменные:
- X — все фичи в виде датафрейма пандаса.
- y — целевая переменная в виде пандасовской серии.
Сохранять X и y внутри модели.
Записать в переменную train_size размер тренировочной выборки (X) в виде кортежа:
(количество_строк, количество_столбцов)
Проверка

Входные данные: несколько наборов тренировочных данных
Выходные данные: значение переменной train_size

In [ ]:
class MyKNNReg():
    def __init__(self, k=3):
        self.k = k
        self.X = None
        self.y = None
        self.train_size = None

    def __str__(self):
        params = (', ').join(f'{key}={value}' for key, value in self.__dict__.items())
        return f'{__class__.__name__} class: {params}'

    def fit(self, X:pd.DataFrame, y:pd.Series):
        self.X = X
        self.y = y
        self.train_size = X.shape

### Предсказание (Евклидовая метрика)

### Метод `predict`

- На вход принимает матрицу фичей в виде датафрейма `pandas`.
- Для каждого объекта тестовой выборки последовательно выполняются следующие шаги:
  1. Вычислить расстояние до каждого объекта из обучающей выборки.
     - Сейчас расстояние будем вычислять по формуле Евклида. Для двух точек с тремя координатами (фичами) Евклидово расстояние считается так:

     $$
     D_{\text{eucl}} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2}
     $$

     где:
     - \( x_1, y_1, z_1 \) – координаты (или иные количественные свойства) первой точки,
     - \( x_2, y_2, z_2 \) – координаты второй точки.
  2. Отобрать \( k \) объектов обучающей выборки, расстояние до которых минимально.
  3. Определить, какой класс наиболее часто встречается (мода) – это и будет классом целевого объекта.
     - Если число объектов в каждом из классов одинаковое, возвращаем класс `1`.

- **Заметка:** Рассматриваем только задачу бинарной классификации (с классами `0` и `1`).
- Вернуть вектор предсказаний.

In [ ]:
class MyKNNReg():
    def __init__(self, k=3):
        self.k = k
        self.X = None
        self.y = None
        self.train_size = None

    def __str__(self):
        params = (', ').join(f'{key}={value}' for key, value in self.__dict__.items())
        return f'{__class__.__name__} class: {params}'

    def fit(self, X:pd.DataFrame, y:pd.Series):
        self.X = X
        self.y = y
        self.train_size = X.shape

    def compute_distances(self, X_test:pd.DataFrame):
        distances = np.sqrt(((self.X.values - X_test.values[:, np.newaxis]) ** 2).sum(axis=2))
        return distances

    def predict(self, X_test:pd.DataFrame):
        distances = self.compute_distances(X_test)
        # отбор соседей с минимальным расстоянием до них
        nearest_neighbors = np.argsort(distances, axis=1)[:, :self.k]
        # классы ближайших соседей
        nearest_labels = self.y.values[nearest_neighbors]
        # Усреднение таргета среди отобранных соседей
        predict_labels = nearest_labels.mean(axis=1)
        return np.array(predict_labels)


### Добавление большего кол-ва вариантов дистанций

Добавьте в класс `MyKNNReg` параметр `metric`, который принимает одно из следующих значений:
- `euclidean`
- `chebyshev`
- `manhattan`
- `cosine`

Значение по умолчанию: `euclidean`.

При обучении и вычислении расстояния между точками должна использоваться соответствующая метрика. Вспомним, как они рассчитываются:

### Евклидово расстояние:
$$
D_{\text{eucl}} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2}
$$

### Расстояние Чебышёва:
$$
D_{\text{cheb}} = \max\left( |x_2 - x_1|, |y_2 - y_1|, |z_2 - z_1| \right)
$$

### Манхэттенское расстояние:
$$
D_{\text{manh}} = |x_2 - x_1| + |y_2 - y_1| + |z_2 - z_1|
$$

### Косинусное расстояние:
$$
D_{\text{cos}} = 1 - \frac{x_1 x_2 + y_1 y_2 + z_1 z_2}{\sqrt{x_1^2 + y_1^2 + z_1^2} \cdot \sqrt{x_2^2 + y_2^2 + z_2^2}}
$$


In [ ]:
class MyKNNReg():
    def __init__(self, k=3, metric='euclidean'):
        self.k = k
        self.X = None
        self.y = None
        self.train_size = None
        self.metric = metric

    def __str__(self):
        params = (', ').join(f'{key}={value}' for key, value in self.__dict__.items())
        return f'{__class__.__name__} class: {params}'

    def fit(self, X:pd.DataFrame, y:pd.Series):
        self.X = X
        self.y = y
        self.train_size = X.shape

    def compute_distances(self, X_test:pd.DataFrame):
        if self.metric == 'euclidean':
            distances = np.sqrt(((self.X.values - X_test.values[:, np.newaxis]) ** 2).sum(axis=2))
            return distances
        elif self.metric == 'chebyshev':
            distances = np.max(np.abs(self.X.values - X_test.values[:, np.newaxis]), axis=2)
            return distances
        elif self.metric == 'manhattan':
            distances = np.abs(self.X.values - X_test.values[:, np.newaxis]).sum(axis=2)
            return distances
        elif self.metric == 'cosine':
            X_norm = np.linalg.norm(self.X.values, axis=1)
            X_test_norm = np.linalg.norm(X_test.values, axis=1)[:, np.newaxis]
            cosine_similarity = np.dot(X_test.values, self.X.values.T) / (X_test_norm * X_norm)
            distances = 1 - cosine_similarity
            return distances
        else:
          return f'Ошибка: Введите корректную метрику!'

    def predict(self, X_test:pd.DataFrame):
        distances = self.compute_distances(X_test)
        # отбор соседей с минимальным расстоянием до них
        nearest_neighbors = np.argsort(distances, axis=1)[:, :self.k]
        # классы ближайших соседей
        nearest_labels = self.y.values[nearest_neighbors]
        # Усреднение таргета среди отобранных соседей
        predict_labels = nearest_labels.mean(axis=1)
        return np.array(predict_labels)

### Взвещенный KNN

Добавьте в класс MyKNNReg параметр weight, которые принимает одно из следующих значений:

uniform
rank
distance
Значение по-умолчанию: uniform

Внесите следуюшие изменения в работу алгоритма:

Если weight = uniform работаем как и раньше.
Если weight = rank или distance, то метод predict должен вычислять предсказания с учетом весов соседей. Формулы по которым вычисляются веса соседей приведены на предыдущем шаге.

In [ ]:
class MyKNNReg():
    def __init__(self, k=3, metric='euclidean', weight='uniform'):
        self.k = k
        self.X = None
        self.y = None
        self.train_size = None
        self.metric = metric
        self.weight = weight

    def __str__(self):
        params = (', ').join(f'{key}={value}' for key, value in self.__dict__.items())
        return f'{self.__class__.__name__} class: {params}'

    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.X = X
        self.y = y
        self.train_size = X.shape

    def compute_distances(self, X_test: pd.DataFrame):
        if self.metric == 'euclidean':
            distances = np.sqrt(((self.X.values - X_test.values[:, np.newaxis]) ** 2).sum(axis=2))
        elif self.metric == 'chebyshev':
            distances = np.max(np.abs(self.X.values - X_test.values[:, np.newaxis]), axis=2)
        elif self.metric == 'manhattan':
            distances = np.abs(self.X.values - X_test.values[:, np.newaxis]).sum(axis=2)
        elif self.metric == 'cosine':
            X_norm = np.linalg.norm(self.X.values, axis=1)
            X_test_norm = np.linalg.norm(X_test.values, axis=1)[:, np.newaxis]
            cosine_similarity = np.dot(X_test.values, self.X.values.T) / (X_test_norm * X_norm)
            distances = 1 - cosine_similarity
        else:
            raise ValueError("Ошибка: Введите корректную метрику!")
        return distances

    def compute_labels(self, nearest_labels: np.ndarray, distances: np.ndarray, nearest_neighbors: np.ndarray):
        # Предсказания с учетом весов
        predicted_labels = []
        for i in range(len(nearest_neighbors)):
            if self.weight == 'uniform':
                # Простое среднее значение
                prediction = nearest_labels[i].mean()
            elif self.weight == 'rank':
                # Веса по рангу (обратный ранг)
                ranks = np.arange(1, self.k + 1)  # ранжируем соседей от 1 до k
                weights = 1 / ranks
                weights /= weights.sum()  # нормализация весов
                prediction = np.dot(nearest_labels[i], weights)
            elif self.weight == 'distance':
                # Веса по расстоянию (обратное расстояние)
                neighbor_distances = distances[i, nearest_neighbors[i]]
                weights = 1 / (neighbor_distances)
                weights /= weights.sum()  # нормализация весов
                prediction = np.dot(nearest_labels[i], weights)
            else:
                raise ValueError("Ошибка: Неверное значение параметра weight!")

            predicted_labels.append(prediction)
        return predicted_labels

    def predict(self, X_test: pd.DataFrame):
        distances = self.compute_distances(X_test)
        # Отбор индексов соседей с минимальным расстоянием
        nearest_neighbors = np.argsort(distances, axis=1)[:, :self.k]
        # Извлечение значений таргета ближайших соседей
        nearest_labels = self.y.values[nearest_neighbors]
        # Подсчёт предсказаний с учетом весов
        predict_labels = self.compute_labels(nearest_labels, distances, nearest_neighbors)
        return np.array(predict_labels)